### Definindo condicionais no SQL com o CASE

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Criando a conexão
import sqlite3
con = sqlite3.connect('BaseDados.db')

In [3]:
# Criando um cursor
cur = con.cursor()

In [4]:
# Criando uma função para consultar os dados
def executa_sql(comando):
    cur.execute(comando)
    resultado = cur.fetchall()
    resultado = pd.DataFrame(resultado)
    if resultado.shape[1] > 0:
        resultado.columns = [i[0] for i in cur.description]
    print(resultado.shape)
    display(resultado.head())
    return resultado

In [5]:
# Utilizando o ORDER BY que vimos na última aula
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc')

(15, 4)


,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Bárbara da Cunha,63546,10.0,1
1,Bárbara Freitas,19442,8.0,2
2,Gabriela Costela,21262,8.0,1
3,Júlia Pinto,47086,8.0,2
4,Lívia Jesus,22284,8.0,1


### CASE
- O `CASE` permite definir um condicional na própria consulta SQL
- Ele vai retornar a **primeira condição que for verdadeira**
- Para escrever o case vamos passar no `WHEN` a condição e no `THEN` o valor caso a condição for verdadeira. Além disso sempre precisamos finalizar o case com o `END`
<br><br>
- Vamos definir a seguinte regra para aplicar nesses dados:
    - Nota > 7: aprovado
    - Nota > 5 e qtd_provas = 1: fazer prova 2
    - Nota > 5 e qtd_provas = 2: fazer prova final
    - Nota > 5 e qtd_provas > 2: revisar matéria
    - Nota < 5 e qtd_provas = 1: revisar matéria
    - Nota < 5 e qtd_provas > 1: lista de exercícios

In [8]:
# Vamos utilizar o CASE para colocar apenas a primeira condição
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, \
                            AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas, \
                            (CASE \
                                WHEN AVG(nota_prova) > 7 THEN "Aprovado" \
                            END) as situacao_aluno \
                            FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc')

(15, 5)


,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.0,1,Aprovado
1,Bárbara Freitas,19442,8.0,2,Aprovado
2,Gabriela Costela,21262,8.0,1,Aprovado
3,Júlia Pinto,47086,8.0,2,Aprovado
4,Lívia Jesus,22284,8.0,1,Aprovado


In [9]:
# Visualizando o resultado
display(resultado_sql)

,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.000000,1,Aprovado
1,Bárbara Freitas,19442,8.000000,2,Aprovado
2,Gabriela Costela,21262,8.000000,1,Aprovado
3,Júlia Pinto,47086,8.000000,2,Aprovado
4,Lívia Jesus,22284,8.000000,1,Aprovado
5,Melissa Ribeiro,38438,8.000000,2,Aprovado
6,Maria Eduarda da Rocha,38273,7.666667,3,Aprovado
7,Eloah Aragão,65749,7.500000,4,Aprovado
8,Isabelly Souza,31859,7.000000,1,None
9,Kevin Melo,80515,7.000000,1,None


In [ ]:

Nota > 5 e qtd_provas > 2: revisar matéria
Nota < 5 e qtd_provas = 1: revisar matéria
Nota < 5 e qtd_provas > 1: lista de exercícios

In [19]:
# Adicionando TODAS as condições
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, \
                            AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas, \
                            (CASE \
                                WHEN AVG(nota_prova) > 7 THEN "Aprovado" \
                                WHEN AVG(nota_prova) > 5 AND count(nota_prova) = 1 THEN "Fazer prova 2" \
                                WHEN AVG(nota_prova) > 5 AND count(nota_prova) = 2 THEN "Fazer prova final" \
                                WHEN AVG(nota_prova) > 5 AND count(nota_prova) > 2 THEN "Revisar matéria" \
                                WHEN AVG(nota_prova) < 5 AND count(nota_prova) = 1 THEN "Revisar matéria" \
                                WHEN AVG(nota_prova) < 5 AND count(nota_prova) > 1 THEN "Lista de exercícios" \
                                ELSE "Verificar manualmente" \
                            END) as situacao_aluno \
                            FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc')

(15, 5)


,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.0,1,Aprovado
1,Bárbara Freitas,19442,8.0,2,Aprovado
2,Gabriela Costela,21262,8.0,1,Aprovado
3,Júlia Pinto,47086,8.0,2,Aprovado
4,Lívia Jesus,22284,8.0,1,Aprovado


In [20]:
display(resultado_sql)

,nome_aluno,cod_matricula,media_prova,qtd_provas,situacao_aluno
0,Bárbara da Cunha,63546,10.000000,1,Aprovado
1,Bárbara Freitas,19442,8.000000,2,Aprovado
2,Gabriela Costela,21262,8.000000,1,Aprovado
3,Júlia Pinto,47086,8.000000,2,Aprovado
4,Lívia Jesus,22284,8.000000,1,Aprovado
5,Melissa Ribeiro,38438,8.000000,2,Aprovado
6,Maria Eduarda da Rocha,38273,7.666667,3,Aprovado
7,Eloah Aragão,65749,7.500000,4,Aprovado
8,Isabelly Souza,31859,7.000000,1,Fazer prova 2
9,Kevin Melo,80515,7.000000,1,Fazer prova 2
